# Introduction to QAMpy

## A simple QPSK simulation

Here we go through how to construct a basic "simulation" for QPSK equalisation. QAMpy aims to have all the relevant building blocks ready for you. 

### First let us import the relevant modules 

QAMpy provides two sets of APIs:
* The basic API
  
This works with signal objects (a numpy array subclass). Generally it should do all the right things for you and makes a lot of the repetitive tasks very easy. Importantly we will strife to keep this stable so that code written for older versions should still work with newer versions.
* The core or advanced API

The core or advanced API is where all the actual processing code sits. One of the important things to remember when using is that many of the core API functions do not preserve signalobject class, but instead return numpy arrays, so you will need to keep track of this. 

It should in general be always possible to use the basic API for your processing

In [ ]:
from qampy import signals, impairments, equalisation, phaserec, helpers
import numpy as np


In [ ]:
# here we use bokeh for plotting as it is much faster
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()

OK lets start by generating a two polarization QPSK signal

In [ ]:
sig = signals.SignalQAMGrayCoded(4, 2**16, fb=25e9, nmodes=2)

Note the arguments. The *4* indicates 4-QAM, the second number is the number of symbols. *fb* is the symbol rate, here we chose 25 Gbaud, by default this would be one. The *nmodes* keyword gives the number of spatial modes (two for dual-polarization). Most QAMpy functions are "SDM-ready", i.e. everything should *"just work"*, but it has not been tested as extensively. 

Generally functions and classes are well documented. For example to see more options for the SignalQAMGrayCoded signals see:

In [ ]:
help(signals.SignalQAMGrayCoded)

So lets have a look at this signal. 

In [ ]:
def plot_constellation(E):
    fig = figure(title="QPSK signal constellation", output_backend="webgl")
    fig.scatter(E[0].real, E[0].imag, color='red', alpha=0.3, legend="X")
    fig.scatter(E[1].real, E[1].imag, color='blue', alpha=0.3, legend="Y")
    fig.xaxis[0].axis_label = "In-Phase"
    fig.yaxis[0].axis_label = "Quadrature"
    show(fig)

In [ ]:
plot_constellation(sig)

OK not much to see here, as we do not have any noise. So lets change the SNR of the signal.

In [ ]:
sig_noisy = impairments.change_snr(sig, 15) #snr is given in dB

In [ ]:
plot_constellation(sig_noisy)

ok that looks more realistic. OK normally we want to do some pulse-shaping, so we need to oversample. 

In [ ]:
sig_noisy2 = sig_noisy.resample(2*sig_noisy.fb, beta=0.1, renormalise=True)

so whats the sampling frequency now?

In [ ]:
print("%.1f GHz"%(sig_noisy2.fs/1e9))
print(sig_noisy2[:,::2].cal_ser())

In [ ]:
plot_constellation(sig_noisy2)

ok this looks like a mess now, but this is expected considering that we are oversampling. The beauty is however that the usual numpy indexing tricks still work.

In [ ]:
plot_constellation(sig_noisy2[:,::2])

so lets add some phase noise as well while we are add it

In [ ]:
sig_noisy2 = impairments.apply_phase_noise(sig_noisy, 100e3)

In [ ]:
plot_constellation(sig_noisy2)

and finally maybe some PMD

In [ ]:
sig_noisy2 = impairments.apply_PMD(sig_noisy2, np.pi/5., 40e-12)

In [ ]:
plot_constellation(sig_noisy2)

### so let's get it back

In [ ]:
wxy, err = equalisation.equalise_signal(sig_noisy2, 2e-3, Ntaps=17, method="cma")

so did the equaliser converge?

In [ ]:
fig = figure(title="Error", output_backend="webgl")
fig.line(np.arange(err[0].shape[0]), abs(err[0]), color='red', alpha=1, legend="X")
fig.xaxis[0].axis_label = "symbol"
fig.yaxis[0].axis_label = "error"
show(fig)

In [ ]:
fig = figure(title="Error", output_backend="webgl")
fig.line(np.arange(wxy[0].shape[1]), wxy[0][0].real, color='red', alpha=1, legend="hxx")
fig.line(np.arange(wxy[0].shape[1]), wxy[0][1].real, color='blue', alpha=1, legend="hxy")
fig.xaxis[0].axis_label = "tap"
fig.yaxis[0].axis_label = "weight"
show(fig)

In [ ]:
sig_out = equalisation.apply_filter(sig_noisy2, wxy)

In [ ]:
plot_constellation(sig_out)

### Let's do some phase recovery

In [ ]:
sig_out2, ph = phaserec.viterbiviterbi(sig_out, 11)
sig_out2 = helpers.normalise_and_center(sig_out2)
sig_out2 = helpers.dump_edges(sig_out2, 10)

In [ ]:
plot_constellation(sig_out2)

### How good is our signal?

In [ ]:
print("SER = ",sig_out2.cal_ser())
print("BER = ", sig_out2.cal_ber())

### We also have EVM and GMI build in

In [ ]:
print("GMI =", sig_out2.cal_gmi()[0])
print("EVM = ", sig_out2.cal_evm())

## There is other equaliser methods as well. 

* modified CMA.
* several Decision Directed LMS algorithms (see equalisation module for details)

In [ ]:
wxy2, err2 = equalisation.equalise_signal(sig_noisy2, 1e-3, Ntaps=21, method="mcma")

In [ ]:
sig_out3 = equalisation.apply_filter(sig_noisy2, wxy)
sig_out3, ph = phaserec.viterbiviterbi(sig_out3, 11)
sig_out3 = helpers.normalise_and_center(sig_out3)
sig_out3 = helpers.dump_edges(sig_out3, 10)
plot_constellation(sig_out3)
print("SER = ",sig_out2.cal_ser())
print("BER = ", sig_out2.cal_ber())

## Speed

We really wanted to make this fast, so that you can do "live" processing. All the time critical functions are implemented using "cython" and run essentially at C-speed with multi-threading support and a fallback option using numba. We also have some initial GPU processing support, but this is still work in progress (In the future you should be able to select backends quite easily). Let's check this out:

In [ ]:
%%timeit 
sig = signals.SignalQAMGrayCoded(4, 10**5, fb=25e9, nmodes=2)
sig = sig.resample(sig.fb*2, beta=0.1, renormalise=True)
sig = impairments.apply_phase_noise(sig, 100e3)
sig = impairments.apply_PMD(sig, np.pi/5.6, 60e-12)
sig = impairments.change_snr(sig, 15)
wxy, err = equalisation.equalise_signal(sig, 1e-3, Ntaps=21, method="mcma")
sig_out3 = equalisation.apply_filter(sig, wxy)
sig_out3, ph = phaserec.viterbiviterbi(sig_out3, 11)
sig_out3 = helpers.normalise_and_center(sig_out3)
sig_out3 = helpers.dump_edges(sig_out3, 10)


The equaliser is actually much faster than a lot of the other things

In [ ]:
%%timeit
equalisation.equalise_signal(sig, 1e-3, Ntaps=21, method="mcma")

# This is even more impressive if you put it in a loop

In [ ]:
# setup some data first because we want to demo the time of the equaliser
sigs = []
for i in range(5):
    s = signals.SignalQAMGrayCoded(4, 10**5, nmodes=2, fb=40e9)
    s = s.resample(2*s.fb, beta=0.1, renormalise=True)
    s = impairments.change_snr(s, 13)
    s = impairments.apply_PMD(s, np.pi/5.5, 50e-12)
    sigs.append(s)
fig = figure(title="QPSK signal constellation", output_backend="webgl")
Xp = fig.scatter(x=s[0].real, y=s[0].imag,  color='blue', alpha=0.3)
Yp = fig.scatter(x=s[1].real, y=s[1].imag,  color='red', alpha=0.3)
handle=show(fig, notebook_handle=True)

for i in range(50):
    wxy, err = equalisation.equalise_signal(sigs[i%5], 2e-3, Ntaps=17, method="mcma", adaptive_stepsize=True)
    sout = equalisation.apply_filter(sigs[i%5], wxy)
    sout = helpers.normalise_and_center(sout)
    # note plotting is a significant bottleneck if we plot all points!
    Xp.data_source.data["x"] = sout[0][::10].real
    Xp.data_source.data["y"] = sout[0][::10].imag
    Yp.data_source.data["x"] = sout[1][::10].real
    Yp.data_source.data["y"] = sout[1][::10].imag
    push_notebook()

## How can we calculate the SER without the symbols?


In [ ]:
sig

## What are these objects?

  * Signals are numpy array subclasses

In [ ]:
isinstance(sig, np.ndarray)

  * a lot of nice build in functionality

In [ ]:
print(sig.cal_ser())
print(sig.cal_gmi())
print(sig.fb)
print(sig.fs)

  * all (almost) all array functions work

In [ ]:
s2= sig*2+np.ones(sig.shape)
print(type(s2))

   * they remember their original symbols

In [ ]:
s2 = sig.resample(2*sig.fb, beta=0.2)
np.all(s2.symbols ==  sig.symbols)

and much more (see the documentation)